In [1]:
!pip install tensorflow==2.15.0
!pip install keras==2.15.0


!pip install imageio
!pip install opencv-python

In [2]:
# import neccessary libraries
from keras.layers import UpSampling2D, Reshape, Activation, Conv2D, BatchNormalization, LeakyReLU, Input, Flatten, multiply, Embedding
from keras.optimizers import Adam
from keras.models import Sequential, Model
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd

from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import numpy as np
from keras.layers import ZeroPadding2D
from keras.layers import Concatenate


from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist

import time


import cv2
from tensorflow.keras.optimizers import Adam
from keras.optimizers import Adam as LegacyAdam
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from keras.models import Sequential, Model
from keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten, Input
from keras.optimizers import Adam as LegacyAdam
import tensorflow as tf  


import glob
import imageio
from tensorflow.keras import layers
from IPython import display

In [3]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
physical_devices = tf.config.list_physical_devices("GPU")


In [4]:
# Function to load images from a directory

# Define the paths
glioma_tumor_path = "C:\\Users\\net pc\\Desktop\\Thesis Paper 3\\Brain tumor dataset\\Testing\\glioma_tumor"



def load_and_resize_images(folder_path, target_size=(64, 64)):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            img_path = os.path.join(folder_path, filename)
            img = Image.open(img_path)
            img = img.resize(target_size, Image.ANTIALIAS)
            img_array = np.array(img)
            images.append(img_array)
    return np.array(images)

# Assuming glioma_tumor is the path as previously defined
glioma_tumor_images = load_and_resize_images(glioma_tumor_path)



print("glioma_tumor image shape:", glioma_tumor_images.shape)

C:\Users\net pc\AppData\Local\Temp\ipykernel_20768\2614605422.py:14: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize(target_size, Image.ANTIALIAS)


glioma_tumor image shape: (100, 64, 64, 3)


In [5]:

glioma_tumor_images = glioma_tumor_images.reshape(glioma_tumor_images.shape[0], 64, 64, 3).astype('float32')
glioma_tumor_images = (glioma_tumor_images - 127.5) / 127.5  # Normalize the images to [-1, 1]

In [6]:
batch_size = 32  # Choose your desired batch size
glioma_tumor_images = tf.data.Dataset.from_tensor_slices(glioma_tumor_images).batch(batch_size)





In [7]:
import matplotlib.pyplot as plt

# Assuming glioma_tumor_images is the array containing the loaded images
# Choose an index to view a specific image
index = 0

# Display the image
plt.imshow(glioma_tumor_images[index])
plt.axis('off')
plt.show()


TypeError: '_BatchDataset' object is not subscriptable

In [ ]:
print(" image size:", glioma_tumor_images)

In [ ]:
label_mode=None
batch_size=32
shuffle=True
seed=None
validation_split=None
(lambda x: x / 255.0)





In [ ]:
discriminator = keras.Sequential(
    [
        keras.Input(shape=(64, 64, 3)),
        layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(0.2),
        layers.Flatten(),
        layers.Dropout(0.2),
        layers.Dense(1, activation="sigmoid"),
    ],
    name="discriminator",
)
print(discriminator.summary())

In [ ]:
latent_dim = 128
generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        layers.Dense(8 * 8 * 128),
        layers.Reshape((8, 8, 128)),
        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(3, kernel_size=5, padding="same", activation="sigmoid"),
    ],
    name="generator",
)
generator.summary()

In [ ]:
opt_gen = keras.optimizers.Adam(1e-4)
opt_disc = keras.optimizers.Adam(1e-4)
loss_fn = keras.losses.BinaryCrossentropy()

In [ ]:
import os
from tqdm import tqdm

# Ensure the directory exists
os.makedirs("generated_images", exist_ok=True)

# Training loop
for epoch in range(2):
    for idx, real in enumerate(tqdm(glioma_tumor_images)):
        batch_size = real.shape[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))
        fake = generator(random_latent_vectors)

        if idx % 100 == 0:
            fig = plt.figure(figsize=(16, 16))  # Adjust figure size
            for i in range(1):  # Change to 16 if you want 4x4 grid
                plt.subplot(4, 4, i+1)  # Adjust subplot parameters for 4x4 grid
                plt.imshow(fake[i, :, :, 0] * 127.5 + 127.5, cmap='gray')  # Adjust colormap and scaling
                plt.axis('off')
            plt.savefig(f"generated_images/generated_img{epoch}_{idx}.png", bbox_inches='tight')  # Save figure with tight bounding box
            plt.close(fig)  # Close the figure to release memory

        ### Train Discriminator: max log(D(x)) + log(1 - D(G(z))
        with tf.GradientTape() as disc_tape:
            loss_disc_real = loss_fn(tf.ones((batch_size, 1)), discriminator(real))
            loss_disc_fake = loss_fn(tf.zeros((batch_size, 1)), discriminator(fake))
            loss_disc = (loss_disc_real + loss_disc_fake) / 2

        grads = disc_tape.gradient(loss_disc, discriminator.trainable_weights)
        opt_disc.apply_gradients(
            zip(grads, discriminator.trainable_weights)
        )

        ### Train Generator min log(1 - D(G(z)) <-> max log(D(G(z))
        with tf.GradientTape() as gen_tape:
            fake = generator(random_latent_vectors)
            output = discriminator(fake)
            loss_gen = loss_fn(tf.ones((batch_size, 1)), output)

        grads = gen_tape.gradient(loss_gen, generator.trainable_weights)
        opt_gen.apply_gradients(
            zip(grads, generator.trainable_weights)
        )
